In [ ]:
# Import all the libraries required

import soccerdata as sd
import pandas as pd
from tqdm import tqdm
import ScraperFC as sfc
import traceback

import warnings
import numpy as np
import socceraction.spadl as spadl

# from pyspark.sql import SparkSession
# from pyspark.sql.types import StructType, StructField, StringType
# import pyspark.pandas as ps

In [ ]:
import datetime

In [ ]:
## Create a connection to Snowflake
from sqlalchemy import create_engine
import snowflake.connector

In [ ]:
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
# spark = SparkSession.builder \
#     .master("local") \
#     .appName("Gegenstats") \
#     .getOrCreate()

In [ ]:
scraper = sfc.Understat()
def scrape_league_data(years):
    output_df = pd.DataFrame(columns=['№', 'Team', 'M', 'W', 'D', 'L', 'G', 'GA', 'PTS', 'xG', 'NPxG', 'xGA',
       'NPxGA', 'NPxGD', 'PPDA', 'OPPDA', 'DC', 'ODC', 'xPTS', 'season','league'])
    for league_year in league_years:
        try:
            ooper = scraper.scrape_league_table(year=league_year[1], league=league_year[0])
            season_val = league_year[1]
            ooper['season'] = int(str(int(str(season_val)[2:])-1)+str(season_val)[2:])
            if league_year[0] == 'EPL':
                ooper['league'] = 'ENG-Premier League'
            elif league_year[0] == 'La Liga':
                ooper['league'] = 'ESP-La Liga'
            elif league_year[0] == 'Bundesliga':
                ooper['league'] = 'GER-Bundesliga'
            elif league_year[0] == 'Serie A':
                ooper['league'] = 'ITA-Serie A'
            elif league_year[0] == 'Ligue 1':
                ooper['league'] = 'FRA-Ligue 1'
            output_df = pd.concat([output_df, ooper])
            
        except Exception as e:
            print(f"An error occurred for the year {league_years}: {e}")
            traceback.print_exc()
    scraper.close()
    return output_df

# league_years = [('La Liga', 2024), ('EPL', 2024), ('Bundesliga', 2024), ('Serie A', 2024), ('Ligue 1', 2024)]   # Replace with your list of years
league_years = [('EPL', 2024), ('La Liga', 2024), ('Bundesliga', 2024), ('Serie A', 2024), ('Ligue 1', 2024)] 
out = scrape_league_data(league_years)

In [ ]:
understat_table = out[['league','Team','season', 'W', 'D', 'L','G','GA', 'xG', 'xGA','NPxG','NPxGA','PPDA',
                       'OPPDA','PTS','xPTS']]

In [ ]:
understat_table.loc[understat_table['Team'] == 'Arminia Bielefeld', 'Team'] = 'Arminia'
understat_table.loc[understat_table['Team'] == 'Borussia Dortmund', 'Team'] = 'Dortmund'
understat_table.loc[understat_table['Team'] == 'Eintracht Frankfurt', 'Team'] = 'Eint Frankfurt'
understat_table.loc[understat_table['Team'] == 'Greuther Fuerth', 'Team'] = 'Greuther Fürth'
understat_table.loc[understat_table['Team'] == 'Hertha Berlin', 'Team'] = 'Hertha BSC'
understat_table.loc[understat_table['Team'] == 'FC Cologne', 'Team'] = 'Köln'
understat_table.loc[understat_table['Team'] == 'Bayer Leverkusen', 'Team'] = 'Leverkusen'
understat_table.loc[understat_table['Team'] == 'Borussia M.Gladbach', 'Team'] = "M'Gladbach"
understat_table.loc[understat_table['Team'] == 'RasenBallsport Leipzig', 'Team'] = 'RB Leipzig'
understat_table.loc[understat_table['Team'] == 'VfB Stuttgart', 'Team'] = 'Stuttgart'
understat_table.loc[understat_table['Team'] == 'Darmstadt', 'Team'] = 'Darmstadt 98'
understat_table.loc[understat_table['Team'] == 'FC Heidenheim', 'Team'] = 'Heidenheim'

In [ ]:
understat_table.loc[understat_table['Team'] == 'Alaves', 'Team'] = 'Alavés'
understat_table.loc[understat_table['Team'] == 'Atletico Madrid', 'Team'] = 'Atlético Madrid'
understat_table.loc[understat_table['Team'] == 'Cadiz', 'Team'] = 'Cádiz'
understat_table.loc[understat_table['Team'] == 'Real Betis', 'Team'] = 'Betis'
understat_table.loc[understat_table['Team'] == 'Real Valladolid', 'Team'] = "Valladolid"
understat_table.loc[understat_table['Team'] == 'Almeria', 'Team'] = 'Almería'

understat_table.loc[understat_table['Team'] == 'Verona', 'Team'] = 'Hellas Verona'
understat_table.loc[understat_table['Team'] == 'Milan', 'Team'] = 'AC Milan'


understat_table.loc[understat_table['Team'] == 'Paris Saint Germain', 'Team'] = "Paris S-G"
understat_table.loc[understat_table['Team'] == 'Saint-Etienne', 'Team'] = "Saint-Étienne"

understat_table.loc[understat_table['Team'] == 'Manchester United', 'Team'] = 'Manchester Utd'
understat_table.loc[understat_table['Team'] == 'Newcastle United', 'Team'] = 'Newcastle Utd'
understat_table.loc[understat_table['Team'] == 'Tottenham Hotspur', 'Team'] = 'Tottenham'
understat_table.loc[understat_table['Team'] == 'Wolverhampton Wanderers', 'Team'] = 'Wolves'
understat_table.loc[understat_table['Team'] == 'Nottingham Forest', 'Team'] = "Nott'ham Forest"
understat_table.loc[understat_table['Team'] == 'Leicester', 'Team'] = 'Leicester City'
understat_table.loc[understat_table['Team'] == 'Leeds', 'Team'] = 'Leeds United'
understat_table.loc[understat_table['Team'] == 'Sheffield United', 'Team'] = "Sheffield Utd"
understat_table.loc[understat_table['Team'] == 'Luton', 'Team'] = 'Luton Town'
understat_table.loc[understat_table['Team'] == 'Norwich', 'Team'] = 'Norwich City'

In [ ]:
understat_table.sort_values(by=['league','season','Team'], ascending=True, inplace=True)
understat_table.reset_index(drop=True, inplace=True)

In [ ]:
understat_table.rename(columns={'Team':'TEAM_NAME', 'W':'TEAM_WINS', 'D':'TEAM_DRAWS', 'league':'COMPETITION',
                                'L': 'TEAM_LOSSES', 'xPTS':'TEAM_XPTS', 'PTS':'TEAM_PTS', 'season':'SEASON',
                                'NPxGA':'NPXG_AGAINST', 'xGA':'XG_AGAINST'}, inplace=True)

In [ ]:
understat_table.set_index(['COMPETITION', 'SEASON', 'TEAM_NAME'], inplace=True)

Get the match data from Snowflake so that we only scrape for matches we don't have yet 

In [ ]:
fbref = sd.FBref(leagues=['ENG-Premier League', 'GER-Bundesliga', "ESP-La Liga",
                          'ITA-Serie A', 'FRA-Ligue 1'], seasons=["2324"], no_cache=True)
# fbref = sd.FBref(leagues=["FRA-Ligue 1"], seasons=["2324"], no_cache=True)
fbref_team_standard = fbref.read_team_season_stats(stat_type='standard')
# fbref_team_standard_oppo = fbref.read_team_season_stats(stat_type='standard', opponent_stats=True)
fbref_team_shooting = fbref.read_team_season_stats(stat_type='shooting')
fbref_team_shooting_oppo = fbref.read_team_season_stats(stat_type='shooting', opponent_stats=True)
fbref_team_passing = fbref.read_team_season_stats(stat_type='passing')
fbref_team_passing_oppo = fbref.read_team_season_stats(stat_type='passing', opponent_stats=True)
fbref_team_defense = fbref.read_team_season_stats(stat_type='defense')
fbref_team_possession = fbref.read_team_season_stats(stat_type='possession')
fbref_team_misc = fbref.read_team_season_stats(stat_type='misc')
fbref_keeper = fbref.read_team_season_stats(stat_type='keeper')
fbref_schedule = fbref.read_schedule()
# fbref_lineups = fbref.read_lineup()

In [ ]:
fbref_schedule.loc[fbref_schedule['home_team'] == 'Milan', 'home_team'] = 'AC Milan'
fbref_schedule.loc[fbref_schedule['away_team'] == 'Milan', 'away_team'] = 'AC Milan'

In [ ]:
# Convert the date column to datetime format
fbref_schedule['date'] = pd.to_datetime(fbref_schedule['date'])
# fbref_schedule = fbref_schedule[fbref_schedule['date'] < datetime.datetime(2023, 12, 20)]

fbref_schedule = fbref_schedule[fbref_schedule['date'] < datetime.datetime.now().strftime('%Y-%m-%d')]

In [ ]:
SNOWFLAKE_USER = 'kbharaj3'
SNOWFLAKE_PASSWORD = 'Snowfl@key0014'
SNOWFLAKE_ACCOUNT = 'qx25653.ca-central-1.aws'
SNOWFLAKE_WAREHOUSE = 'FOOTY_STORE'
SNOWFLAKE_DATABASE = 'GEGENSTATS'
SNOWFLAKE_SCHEMA = 'TABLES'

In [ ]:
conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE,
    schema=SNOWFLAKE_SCHEMA
)

In [ ]:
cursor = conn.cursor()

In [ ]:
def fetch_data(cursor, query):
    cursor.execute(query)
    rows = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    return pd.DataFrame(rows, columns=column_names)

In [ ]:
df_matches = fetch_data(cursor, 'SELECT * FROM MATCHES')

In [ ]:
df_matches.shape

In [ ]:
# df_matches = df_matches[df_matches['SEASON'] == 2324]

In [ ]:
matches_to_scarpe = set(fbref_schedule['game_id']) - set(df_matches['MATCH_ID'])
matches_to_scarpe = list(matches_to_scarpe)

In [ ]:
len(matches_to_scarpe)

In [ ]:
# matches_to_scarpe = list(fbref_schedule['game_id'])

In [ ]:
# ## Create empty dataframe with 
# pd.DataFrame(columns=['jersey_number', 'player', 'team', 'is_starter', 'position', 'minutes_played'], index=['league', 'season',
#                                                                                                               'game'])

In [ ]:
failed_match_ids = []

In [ ]:
for i, val in tqdm(enumerate(matches_to_scarpe)):
    if i == 0:
        fbref_lineups = fbref.read_lineup(match_id=val)
    else:
        try:
            fbref_lineups = pd.concat([fbref_lineups, fbref.read_lineup(match_id=val)])
        except:
            failed_match_ids.append(val)

In [ ]:
fbref_lineups.to_csv('fbref_lineups_hold.csv')
# fbref_lineups = pd.read_csv('fbref_lineups_hold.csv')

In [ ]:
# fbref_lineups.set_index(['league', 'season', 'game'], inplace=True)

In [ ]:
failed_match_ids

In [ ]:
fbref_lineups.loc[fbref_lineups['team'] == 'Brighton & Hove Albion', 'team'] = 'Brighton'
fbref_lineups.loc[fbref_lineups['team'] == 'Manchester United', 'team'] = 'Manchester Utd'
fbref_lineups.loc[fbref_lineups['team'] == 'Newcastle United', 'team'] = 'Newcastle Utd'
fbref_lineups.loc[fbref_lineups['team'] == 'Nottingham Forest', 'team'] = "Nott'ham Forest"
fbref_lineups.loc[fbref_lineups['team'] == 'Sheffield United', 'team'] = "Sheffield Utd"
fbref_lineups.loc[fbref_lineups['team'] == 'Tottenham Hotspur', 'team'] = 'Tottenham'
fbref_lineups.loc[fbref_lineups['team'] == 'West Ham United', 'team'] = 'West Ham'
fbref_lineups.loc[fbref_lineups['team'] == 'Wolverhampton Wanderers', 'team'] = 'Wolves'

In [ ]:
fbref_lineups.replace({"Jarrell Quansah": "Jarell Quansah"}, inplace=True)
fbref_lineups.replace({"Omotayo Adaramola": "Tayo Adaramola"}, inplace=True)
fbref_lineups.replace({"Cristóbal Montiel": "Luis Montiel"}, inplace=True)

In [ ]:
# fbref_lineups.reset_index(inplace=True)
# fbref_lineups.loc[fbref_lineups['team'] == 'Real Betis', 'team'] = 'Betis'
# fbref_lineups.set_index(['league', 'season', 'game'], inplace=True)

In [ ]:
fbref_team_standard = fbref_team_standard.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
# fbref_team_standard_oppo = fbref_team_standard_oppo.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_shooting = fbref_team_shooting.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_shooting_oppo = fbref_team_shooting_oppo.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_passing = fbref_team_passing.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_passing_oppo = fbref_team_passing_oppo.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_defense = fbref_team_defense.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_possession = fbref_team_possession.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_misc = fbref_team_misc.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_keeper = fbref_keeper.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_schedule = fbref_schedule.rename_axis(index={'season': 'SEASON', 'league': 'COMPETITION'})
fbref_lineups = fbref_lineups.rename_axis(index={'season': 'SEASON', 'league': 'COMPETITION'})
fbref_lineups.rename({'team': 'TEAM_NAME'}, axis=1, inplace=True)
# fbref_lineups.rename({'season': 'SEASON'}, axis=1, inplace=True)

In [ ]:
## Get rid of incomplete games (Thank God Tom Lockyear is good <3)
fbref_schedule = fbref_schedule.dropna(subset=['home_xg'])

## Get rid of 'vs ' in the TEAM_NAME column
fbref_team_passing_oppo.reset_index(inplace=True)
fbref_team_shooting_oppo.reset_index(inplace=True)

In [ ]:
fbref_schedule = fbref_schedule.dropna(subset=['week'])

In [ ]:
fbref_team_passing_oppo['TEAM_NAME'] = fbref_team_passing_oppo['TEAM_NAME'].str.replace('vs ', '')
fbref_team_shooting_oppo['TEAM_NAME'] = fbref_team_shooting_oppo['TEAM_NAME'].str.replace('vs ', '')

fbref_team_passing_oppo.set_index(['COMPETITION', 'SEASON', 'TEAM_NAME'], inplace=True)
fbref_team_shooting_oppo.set_index(['COMPETITION', 'SEASON', 'TEAM_NAME'], inplace=True)

In [ ]:
def make_season_integer(df):
    indexes = df.index.names
    df = df.reset_index()
    df['SEASON'] = df['SEASON'].astype(int)
    df = df.set_index(indexes)
    return df

fbref_team_standard = make_season_integer(fbref_team_standard)
# fbref_team_standard_oppo = make_season_integer(fbref_team_standard_oppo)
fbref_team_shooting = make_season_integer(fbref_team_shooting)
fbref_team_shooting_oppo = make_season_integer(fbref_team_shooting_oppo)
fbref_team_passing = make_season_integer(fbref_team_passing)
fbref_team_passing_oppo = make_season_integer(fbref_team_passing_oppo)
fbref_team_defense = make_season_integer(fbref_team_defense)
fbref_team_possession = make_season_integer(fbref_team_possession)
fbref_team_misc = make_season_integer(fbref_team_misc)
fbref_schedule = make_season_integer(fbref_schedule)
fbref_keeper = make_season_integer(fbref_keeper)
fbref_lineups = make_season_integer(fbref_lineups)

In [ ]:
def name_corrector(df):
    indexes = df.index.names
    df = df.reset_index()
    # df['SEASON'] = df['SEASON'].astype(int)
    df.loc[df['TEAM_NAME'] == 'Milan', 'TEAM_NAME'] = 'AC Milan'
    df = df.set_index(indexes)
    return df

fbref_team_standard = name_corrector(fbref_team_standard)
# fbref_team_standard_oppo = name_corrector(fbref_team_standard_oppo)
fbref_team_shooting = name_corrector(fbref_team_shooting)
fbref_team_shooting_oppo = name_corrector(fbref_team_shooting_oppo)
fbref_team_passing = name_corrector(fbref_team_passing)
fbref_team_passing_oppo = name_corrector(fbref_team_passing_oppo)
fbref_team_defense = name_corrector(fbref_team_defense)
fbref_team_possession = name_corrector(fbref_team_possession)
fbref_team_misc = name_corrector(fbref_team_misc)
# fbref_schedule = name_corrector(fbref_schedule)
fbref_keeper = name_corrector(fbref_keeper)

In [ ]:
team_matches_played = fbref_team_standard['Playing Time'].MP
team_goals_scored = understat_table.G
team_npxG_90 = fbref_team_standard['Per 90 Minutes'].npxG
team_goals_conceded = understat_table.GA
# team_xG_against_90 = fbref_team_standard_oppo['Per 90 Minutes'].xG
team_shots = fbref_team_shooting['Standard'].Sh
team_shotsOT = fbref_team_shooting['Standard'].SoT
team_pass_completed = fbref_team_passing["Total"].Cmp
team_pass_attempted = fbref_team_passing["Total"].Att
team_tackles = fbref_team_defense.Tackles.Tkl
team_tackles_won = fbref_team_defense.Tackles.TklW
team_takeons_attempted = fbref_team_possession['Take-Ons'].Att
team_takeons_completed = fbref_team_possession['Take-Ons'].Succ
team_crossesintoPA = fbref_team_passing['CrsPA']
team_fouls_against = fbref_team_misc['Performance']['Fld']
team_fouls_made = fbref_team_misc['Performance']['Fls']
team_interceptions = fbref_team_defense['Int']
team_blocks_shots = fbref_team_defense['Blocks'].Sh
team_blocks_pass = fbref_team_defense['Blocks'].Pass
team_clearances = fbref_team_defense.Clr
team_aerials_won = fbref_team_misc['Aerial Duels'].Won
team_aerials_lost = fbref_team_misc['Aerial Duels'].Lost
team_shots_faced = fbref_team_shooting_oppo['Standard'].Sh

In [ ]:
team_recoveries= fbref_team_misc['Performance']['Recov']

In [ ]:
teams_poss_won = team_interceptions + team_tackles_won + team_recoveries + team_blocks_shots + \
    team_blocks_pass + team_aerials_won
team_OPPDA = understat_table['OPPDA']
team_final_3rd_passes_against =  fbref_team_passing_oppo['1/3']
team_cleansheets = fbref_keeper['Performance']['CS']

In [ ]:
team_names = fbref_team_standard.reset_index()[['TEAM_NAME','url']]
team_names['TEAM_FBREF_ID'] = team_names['url'].apply(lambda x: x.split('/')[3])      ## Extracting team_id from URL
team_names.drop(columns=['url'], inplace=True)
team_names.columns = team_names.columns.droplevel(1)

In [ ]:
team_names['TEAM_FBREF_ID'] = team_names['TEAM_FBREF_ID'].astype('string')

In [ ]:
team_names.drop_duplicates(inplace=True)

## WhoScored Data

In [ ]:
ws = sd.WhoScored(leagues=['ENG-Premier League', 'GER-Bundesliga', "ESP-La Liga",
                          'ITA-Serie A', 'FRA-Ligue 1'], seasons=["2324"])
# ws = sd.WhoScored(leagues=['FRA-Ligue 1'], seasons=["2324"])

In [ ]:
ws_schedule = ws.read_schedule()

In [ ]:
# ws_schedule = pd.read_csv('ws_schedule.csv')

In [ ]:
ws_schedule.to_csv('ws_schedule.csv')

In [ ]:
ws_schedule = ws_schedule[ws_schedule['date'] < datetime.datetime.now().strftime('%Y-%m-%d')]

In [ ]:
fbref_schedule = fbref_schedule.dropna(subset=['home_xg'])

In [ ]:
ws_schedule = ws_schedule.reset_index().rename({'league':"COMPETITION", "season":"SEASON"}, axis=1)
ws_schedule.set_index(['COMPETITION', 'SEASON', 'game'], inplace=True)

In [ ]:
ws_schedule.loc[ws_schedule['home_team'] == 'Manchester United', 'home_team'] = 'Manchester Utd'
ws_schedule.loc[ws_schedule['away_team'] == 'Manchester United', 'away_team'] = 'Manchester Utd'
ws_schedule.loc[ws_schedule['home_team'] == 'Newcastle United', 'home_team'] = 'Newcastle Utd'
ws_schedule.loc[ws_schedule['away_team'] == 'Newcastle United', 'away_team'] = 'Newcastle Utd'
ws_schedule.loc[ws_schedule['home_team'] == 'Newcastle', 'home_team'] = 'Newcastle Utd'
ws_schedule.loc[ws_schedule['away_team'] == 'Newcastle', 'away_team'] = 'Newcastle Utd'
ws_schedule.loc[ws_schedule['home_team'] == 'Tottenham Hotspur', 'home_team'] = 'Tottenham'
ws_schedule.loc[ws_schedule['away_team'] == 'Tottenham Hotspur', 'away_team'] = 'Tottenham'
ws_schedule.loc[ws_schedule['home_team'] == 'Wolverhampton Wanderers', 'home_team'] = 'Wolves'
ws_schedule.loc[ws_schedule['away_team'] == 'Wolverhampton Wanderers', 'away_team'] = 'Wolves'
ws_schedule.loc[ws_schedule['home_team'] == 'Wolverhampton', 'home_team'] = 'Wolves'
ws_schedule.loc[ws_schedule['away_team'] == 'Wolverhampton', 'away_team'] = 'Wolves'
ws_schedule.loc[ws_schedule['home_team'] == 'Nottingham Forest', 'home_team'] = "Nott'ham Forest"
ws_schedule.loc[ws_schedule['away_team'] == 'Nottingham Forest', 'away_team'] = "Nott'ham Forest"
ws_schedule.loc[ws_schedule['home_team'] == 'Leicester', 'home_team'] = "Leicester City"
ws_schedule.loc[ws_schedule['away_team'] == 'Leicester', 'away_team'] = "Leicester City"
ws_schedule.loc[ws_schedule['home_team'] == 'Leeds', 'home_team'] = "Leeds United"
ws_schedule.loc[ws_schedule['away_team'] == 'Leeds', 'away_team'] = "Leeds United"
ws_schedule.loc[ws_schedule['home_team'] == 'Sheffield United', 'home_team'] = "Sheffield Utd"
ws_schedule.loc[ws_schedule['away_team'] == 'Sheffield United', 'away_team'] = "Sheffield Utd"
ws_schedule.loc[ws_schedule['home_team'] == 'Luton', 'home_team'] = "Luton Town"
ws_schedule.loc[ws_schedule['away_team'] == 'Luton', 'away_team'] = "Luton Town"
ws_schedule.loc[ws_schedule['home_team'] == 'Norwich', 'home_team'] = "Norwich City"
ws_schedule.loc[ws_schedule['away_team'] == 'Norwich', 'away_team'] = "Norwich City"

ws_schedule.loc[ws_schedule['home_team'] == 'Atletico Madrid', 'home_team'] = 'Atlético Madrid'
ws_schedule.loc[ws_schedule['away_team'] == 'Atletico Madrid', 'away_team'] = 'Atlético Madrid'
ws_schedule.loc[ws_schedule['home_team'] == 'Cadiz', 'home_team'] = 'Cádiz'
ws_schedule.loc[ws_schedule['away_team'] == 'Cadiz', 'away_team'] = 'Cádiz'
ws_schedule.loc[ws_schedule['home_team'] == 'Deportivo Alaves', 'home_team'] = 'Alavés'
ws_schedule.loc[ws_schedule['away_team'] == 'Deportivo Alaves', 'away_team'] = 'Alavés'
ws_schedule.loc[ws_schedule['home_team'] == 'Real Betis', 'home_team'] = 'Betis'
ws_schedule.loc[ws_schedule['away_team'] == 'Real Betis', 'away_team'] = 'Betis'
ws_schedule.loc[ws_schedule['home_team'] == 'Deportivo Alaves', 'home_team'] = 'Alavés'
ws_schedule.loc[ws_schedule['away_team'] == 'Deportivo Alaves', 'away_team'] = 'Alavés'
ws_schedule.loc[ws_schedule['home_team'] == 'Real Valladolid', 'home_team'] = 'Valladolid'
ws_schedule.loc[ws_schedule['away_team'] == 'Real Valladolid', 'away_team'] = 'Valladolid'
ws_schedule.loc[ws_schedule['home_team'] == 'Almeria', 'home_team'] = 'Almería'
ws_schedule.loc[ws_schedule['away_team'] == 'Almeria', 'away_team'] = 'Almería'

ws_schedule.loc[ws_schedule['home_team'] == 'Arminia Bielefeld', 'home_team'] = 'Arminia'
ws_schedule.loc[ws_schedule['away_team'] == 'Arminia Bielefeld', 'away_team'] = 'Arminia'
ws_schedule.loc[ws_schedule['home_team'] == 'Bayer Leverkusen', 'home_team'] = 'Leverkusen'
ws_schedule.loc[ws_schedule['away_team'] == 'Bayer Leverkusen', 'away_team'] = 'Leverkusen'
ws_schedule.loc[ws_schedule['home_team'] == 'Borussia Dortmund', 'home_team'] = 'Dortmund'
ws_schedule.loc[ws_schedule['away_team'] == 'Borussia Dortmund', 'away_team'] = 'Dortmund'
ws_schedule.loc[ws_schedule['home_team'] == 'Borussia M.Gladbach', 'home_team'] = "M'Gladbach"
ws_schedule.loc[ws_schedule['away_team'] == 'Borussia M.Gladbach', 'away_team'] = "M'Gladbach"
ws_schedule.loc[ws_schedule['home_team'] == 'Eintracht Frankfurt', 'home_team'] = "Eint Frankfurt"
ws_schedule.loc[ws_schedule['away_team'] == 'Eintracht Frankfurt', 'away_team'] = "Eint Frankfurt"
ws_schedule.loc[ws_schedule['home_team'] == 'Greuther Fuerth', 'home_team'] = "Greuther Fürth"
ws_schedule.loc[ws_schedule['away_team'] == 'Greuther Fuerth', 'away_team'] = "Greuther Fürth"
ws_schedule.loc[ws_schedule['home_team'] == 'FC Koln', 'home_team'] = "Köln"
ws_schedule.loc[ws_schedule['away_team'] == 'FC Koln', 'away_team'] = "Köln"
ws_schedule.loc[ws_schedule['home_team'] == 'Hertha Berlin', 'home_team'] = "Hertha BSC"
ws_schedule.loc[ws_schedule['away_team'] == 'Hertha Berlin', 'away_team'] = "Hertha BSC"
ws_schedule.loc[ws_schedule['home_team'] == 'VfB Stuttgart', 'home_team'] = "Stuttgart"
ws_schedule.loc[ws_schedule['away_team'] == 'VfB Stuttgart', 'away_team'] = "Stuttgart"
ws_schedule.loc[ws_schedule['home_team'] == 'Darmstadt', 'home_team'] = "Darmstadt 98"
ws_schedule.loc[ws_schedule['away_team'] == 'Darmstadt', 'away_team'] = "Darmstadt 98"
ws_schedule.loc[ws_schedule['home_team'] == 'FC Heidenheim', 'home_team'] = "Heidenheim"
ws_schedule.loc[ws_schedule['away_team'] == 'FC Heidenheim', 'away_team'] = "Heidenheim"

ws_schedule.loc[ws_schedule['home_team'] == 'Verona', 'home_team'] = "Hellas Verona"
ws_schedule.loc[ws_schedule['away_team'] == 'Verona', 'away_team'] = "Hellas Verona"

ws_schedule.loc[ws_schedule['home_team'] == 'Paris Saint-Germain', 'home_team'] = "Paris S-G"
ws_schedule.loc[ws_schedule['away_team'] == 'Paris Saint-Germain', 'away_team'] = "Paris S-G"
ws_schedule.loc[ws_schedule['home_team'] == 'AC Ajaccio', 'home_team'] = "Ajaccio"
ws_schedule.loc[ws_schedule['away_team'] == 'AC Ajaccio', 'away_team'] = "Ajaccio"
ws_schedule.loc[ws_schedule['home_team'] == 'Saint-Etienne', 'home_team'] = "Saint-Étienne"
ws_schedule.loc[ws_schedule['away_team'] == 'Saint-Etienne', 'away_team'] = "Saint-Étienne"

In [ ]:
fbref_lineups.loc[fbref_lineups['TEAM_NAME'] == 'Bayer Leverkusen', 'TEAM_NAME'] = 'Leverkusen'
fbref_lineups.loc[fbref_lineups['TEAM_NAME'] == 'Eintracht Frankfurt', 'TEAM_NAME'] = 'Eint Frankfurt'
fbref_lineups.loc[fbref_lineups['TEAM_NAME'] == 'Mönchengladbach', 'TEAM_NAME'] = "M'Gladbach"
fbref_lineups.loc[fbref_lineups['TEAM_NAME'] == 'Milan', 'TEAM_NAME'] = "AC Milan"
fbref_lineups.loc[fbref_lineups['TEAM_NAME'] == 'Internazionale', 'TEAM_NAME'] = "Inter"
fbref_lineups.loc[fbref_lineups['TEAM_NAME'] == 'Paris Saint-Germain', 'TEAM_NAME'] = "Paris S-G"
fbref_lineups.loc[fbref_lineups['TEAM_NAME'] == 'Real Betis', 'TEAM_NAME'] = "Betis"

In [ ]:
ws_schedule.reset_index(inplace=True)
fbref_schedule.reset_index(inplace=True)

In [ ]:
ws_schedule = ws_schedule.rename({'game_id':"ws_game_id"}, axis=1)

In [ ]:
ws_schedule['SEASON'] = ws_schedule['SEASON'].astype('int64')

In [ ]:
sorted(fbref_schedule['home_team'].unique())==sorted(ws_schedule['home_team'].unique())

In [ ]:
fbref_schedule = pd.merge(fbref_schedule, ws_schedule[["COMPETITION", "SEASON", "home_team", "away_team","ws_game_id"]], 
          on=["COMPETITION", "SEASON", "home_team", "away_team"], how='inner')

In [ ]:
loader = ws.read_events(output_fmt='loader')

In [ ]:
game_ids = fbref_schedule[fbref_schedule['game_id'].isin(matches_to_scarpe)]['ws_game_id'].unique()

In [ ]:
ws_players = pd.DataFrame(columns=['game_id', 'team_id', 'player_id', 'player_name', 
                                   'is_starter','minutes_played', 'jersey_number', 'starting_position'])

In [ ]:
ws_teams=pd.DataFrame(columns=['team_id', 'team_name'])

In [ ]:
games_no_players = []

In [ ]:
# game_ids = fbref_schedule['ws_game_id'].unique()
# matches_to_scarpe = fbref_schedule['game_id'].unique()

In [ ]:
game_ids

In [ ]:
for game in tqdm(game_ids):
    try:
        ws_teams = pd.concat([ws_teams, loader.teams(game_id=game)])
        ws_players = pd.concat([ws_players, loader.players(game_id=game)])
    except:
        games_no_players.append(game)

In [ ]:
ws_teams.drop_duplicates(inplace=True)

In [ ]:
ws_teams.reset_index(drop=True, inplace=True)

In [ ]:
sorted(ws_schedule['home_team'].unique()) == sorted(ws_teams['team_name'])

In [ ]:
len(sorted(ws_schedule['home_team'].unique()))

In [ ]:
len(sorted(ws_teams['team_name']))

In [ ]:
ws_teams.loc[ws_teams['team_name'] == 'Arminia Bielefeld', 'team_name'] = 'Arminia'
ws_teams.loc[ws_teams['team_name'] == 'Bayern', 'team_name'] = 'Bayern Munich'
ws_teams.loc[ws_teams['team_name'] == 'Borussia Dortmund', 'team_name'] = 'Dortmund'
ws_teams.loc[ws_teams['team_name'] == 'Borussia M.Gladbach', 'team_name'] = "M'Gladbach"
ws_teams.loc[ws_teams['team_name'] == 'Eintracht Frankfurt', 'team_name'] = "Eint Frankfurt"
ws_teams.loc[ws_teams['team_name'] == 'FC Koln', 'team_name'] = 'Köln'
ws_teams.loc[ws_teams['team_name'] == 'Mainz', 'team_name'] = "Mainz 05"
ws_teams.loc[ws_teams['team_name'] == 'RBL', 'team_name'] = "RB Leipzig"
ws_teams.loc[ws_teams['team_name'] == 'Greuther Fuerth', 'team_name'] = "Greuther Fürth"
ws_teams.loc[ws_teams['team_name'] == 'Hertha Berlin', 'team_name'] = "Hertha BSC"
ws_teams.loc[ws_teams['team_name'] == 'Schalke', 'team_name'] = 'Schalke 04'
ws_teams.loc[ws_teams['team_name'] == 'Darmstadt', 'team_name'] = "Darmstadt 98"
ws_teams.loc[ws_teams['team_name'] == 'FC Heidenheim', 'team_name'] = 'Heidenheim'

ws_teams.loc[ws_teams['team_name'] == 'Luton', 'team_name'] = 'Luton Town'
ws_teams.loc[ws_teams['team_name'] == 'Nottingham Forest', 'team_name'] = "Nott'ham Forest"
ws_teams.loc[ws_teams['team_name'] == 'Sheff Utd', 'team_name'] = "Sheffield Utd"
ws_teams.loc[ws_teams['team_name'] == 'Man Utd', 'team_name'] = "Manchester Utd"
ws_teams.loc[ws_teams['team_name'] == 'Man City', 'team_name'] = "Manchester City"
ws_teams.loc[ws_teams['team_name'] == 'Newcastle', 'team_name'] = "Newcastle Utd"

ws_teams.loc[ws_teams['team_name'] == 'Atletico', 'team_name'] = 'Atlético Madrid'
ws_teams.loc[ws_teams['team_name'] == 'Cadiz', 'team_name'] = 'Cádiz'
ws_teams.loc[ws_teams['team_name'] == 'Deportivo Alaves', 'team_name'] = 'Alavés'
ws_teams.loc[ws_teams['team_name'] == 'Real Betis', 'team_name'] = 'Betis'
ws_teams.loc[ws_teams['team_name'] == 'Almeria', 'team_name'] = 'Almería'
ws_teams.loc[ws_teams['team_name'] == 'Real Valladolid', 'team_name'] = 'Valladolid'

ws_teams.loc[ws_teams['team_name'] == 'Verona', 'team_name'] = 'Hellas Verona'

ws_teams.loc[ws_teams['team_name'] == 'PSG', 'team_name'] = 'Paris S-G'
ws_teams.loc[ws_teams['team_name'] == 'Saint-Etienne', 'team_name'] = 'Saint-Étienne'
ws_teams.loc[ws_teams['team_name'] == 'AC Ajaccio', 'team_name'] = "Ajaccio"

In [ ]:
# WhoScored decide to name Willy Boly with two different names. Idiots
ws_players.replace({"Willy-Arnaud Boly": "Willy Boly"}, inplace=True)
ws_players.replace({"Kepa": "Kepa Arrizabalaga"}, inplace=True)
ws_players.replace({"Yehor Yarmolyuk": "Yehor Yarmoliuk"}, inplace=True)
ws_players.replace({"Vitalii Mykolenko": "Vitaliy Mykolenko"}, inplace=True)
ws_players.replace({"Dan Bentley": "Daniel Bentley"}, inplace=True)
ws_players.replace({"Toti": "Toti Gomes"}, inplace=True)
ws_players.replace({"Joe Hodge": "Joseph Hodge"}, inplace=True)
ws_players.replace({"Jonny Otto": "Jonny"}, inplace=True)
ws_players.replace({"Álvaro Carreras": "Álvaro Fernández"}, inplace=True)
ws_players.replace({"Alisson Becker": "Alisson"}, inplace=True)
ws_players.replace({"Andy Robertson": "Andrew Robertson"}, inplace=True)
ws_players.replace({"Konstantinos Tsimikas": "Kostas Tsimikas"}, inplace=True)
ws_players.replace({"Alphonse Aréola": "Alphonse Areola"}, inplace=True)
ws_players.replace({"Pape Sarr": "Pape Matar Sarr"}, inplace=True)
ws_players.replace({"Yago Alonso": "Yago Santiago"}, inplace=True)

## Snowflake Data Upload

In [ ]:
SNOWFLAKE_USER = 'kbharaj3'
SNOWFLAKE_PASSWORD = 'Snowfl@key0014'
SNOWFLAKE_ACCOUNT = 'qx25653.ca-central-1.aws'
SNOWFLAKE_WAREHOUSE = 'FOOTY_STORE'
SNOWFLAKE_DATABASE = 'GEGENSTATS'
SNOWFLAKE_SCHEMA = 'TABLES'

In [ ]:
conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE,
    schema=SNOWFLAKE_SCHEMA
    )

In [ ]:
cur = conn.cursor()
cur.execute(f"USE WAREHOUSE {SNOWFLAKE_WAREHOUSE}")

In [ ]:
create_schema_sql = f"CREATE SCHEMA IF NOT EXISTS {SNOWFLAKE_SCHEMA}"
cur.execute(create_schema_sql)

In [ ]:
print("The Snowflake warehouse being used is:", SNOWFLAKE_WAREHOUSE)

In [ ]:
cur.execute('SELECT * FROM TEAMS')
team_rows = cur.fetchall()
column_names = [desc[0] for desc in cur.description]
curr_team_names = pd.DataFrame(team_rows, columns=column_names)

In [ ]:
## Below (if not blank) are the teams you need to create LOGOS for
team_names_not_in_curr = team_names[~team_names['TEAM_NAME'].isin(curr_team_names['TEAM_NAME'])]
team_names_not_in_curr

In [ ]:
def upsert_to_snowflake(table_name, dataframe, primary_keys):
    if isinstance(primary_keys, str):
        primary_keys = [primary_keys]

    temp_table_name = f"{table_name}_TEMP"
    success, nchunks, nrows, _ = write_pandas(conn, dataframe, temp_table_name, auto_create_table=True)

    on_condition = ' AND '.join([f"{table_name}.{pk} = {temp_table_name}.{pk}" for pk in primary_keys])

    update_columns = [col for col in dataframe.columns if col not in primary_keys]
    update_sql = ', '.join([f"{table_name}.{col} = {temp_table_name}.{col}" for col in update_columns])

    merge_sql = f"""
    MERGE INTO {table_name} USING {temp_table_name}
    ON {on_condition}
    """

    if update_columns:
        merge_sql += f"""
        WHEN MATCHED THEN
            UPDATE SET
                {update_sql}
        """

    merge_sql += f"""
    WHEN NOT MATCHED THEN
        INSERT ({', '.join(dataframe.columns)})
        VALUES ({', '.join([f"{temp_table_name}.{col}" for col in dataframe.columns])})
    """

    cur.execute(merge_sql)
    cur.execute(f"DROP TABLE IF EXISTS {temp_table_name}")

    print(f"Upsert completed. {nrows} rows processed.")

In [ ]:
ws_teams.rename({'team_name':'TEAM_NAME', 'team_id':'TEAM_WS_ID'}, axis=1, inplace=True)

In [ ]:
team_logos = \
[["Angers", "https://i.imgur.com/dyRj0Cc.png"], ["Bordeaux", "https://i.imgur.com/2NxcpPo.png"],
 ["Brest", "https://i.imgur.com/klKUp8n.png"], ["Clermont Foot", "https://i.imgur.com/7RHafik.png"],
 ["Lens", "https://i.imgur.com/9hfNDts.png"], ["Lille", "https://i.imgur.com/G3MyVe5.png"],
 ["Lorient", "https://i.imgur.com/9k1Kgp6.png"], ["Lyon", "https://i.imgur.com/F0cNDBC.png"],
 ["Marseille", "https://i.imgur.com/jbim9vo.png"], ["Metz", "https://i.imgur.com/UvYK5II.png"],
 ["Monaco", "https://i.imgur.com/HNfIv5Y.png"], ["Montpellier", "https://i.imgur.com/lwG9vvR.png"],
 ["Nantes", "https://i.imgur.com/rxlejD4.png"], ["Nice", "https://i.imgur.com/bIsadPH.png"],
 ["Paris S-G", "https://i.imgur.com/kuN3QSp.png"], ["Reims", "https://i.imgur.com/xULypbI.png"],
 ["Rennes", "https://i.imgur.com/TZq2Hvr.png"], ["Saint-Étienne", "https://i.imgur.com/iIex5w7.png"],
 ["Strasbourg", "https://i.imgur.com/RKf7LuY.png"], ["Troyes", "https://i.imgur.com/6VTNoyL.png"],
 ["Ajaccio", "https://i.imgur.com/FSm4pRb.png"], ["Auxerre", "https://i.imgur.com/mwlqKAg.png"],
 ["Toulouse", "https://i.imgur.com/dvkjqJ2.png"], ["Le Havre", "https://i.imgur.com/YHGxNQP.png"]
 ]

In [ ]:
# team_logos = \
# [["Atalanta", "https://i.imgur.com/SDA0nMH.png"], ["Bologna", "https://i.imgur.com/rVqtfOA.png"],
#  ["Cagliari", "https://i.imgur.com/rbAC6zo.png"], ["Empoli", "https://i.imgur.com/52xSYdX.png"],
#  ["Fiorentina", "https://i.imgur.com/hOQC1jo.png"], ["Genoa", "https://i.imgur.com/KjtIy8U.png"],
#  ["Hellas Verona", "https://i.imgur.com/vhAWZQO.png"], ["Inter", "https://i.imgur.com/JsJ06ke.png"],
#  ["Juventus", "https://i.imgur.com/60XAIJI.png"], ["Lazio", "https://i.imgur.com/SBsGYjm.png"],
#  ["AC Milan", "https://i.imgur.com/1Ze94pt.png"], ["Napoli", "https://i.imgur.com/Ql7LKBZ.png"],
#  ["Roma", "https://i.imgur.com/nhMJAHW.png"], ["Salernitana", "https://i.imgur.com/sYubMBZ.png"],
#  ["Sampdoria", "https://i.imgur.com/MTOvCDt.png"], ["Sassuolo", "https://i.imgur.com/px5mT39.png"],
#  ["Spezia", "https://i.imgur.com/gYVirSf.png"], ["Torino", "https://i.imgur.com/XWmpYeu.png"],
#  ["Udinese", "https://i.imgur.com/Dp0KUmN.png"], ["Venezia", "https://i.imgur.com/VjHBE7d.png"],
#  ["Cremonese", "https://i.imgur.com/r6hDldN.png"], ["Lecce", "https://i.imgur.com/9A4flGu.png"], 
#  ["Monza", "https://i.imgur.com/7bHOIx9.png"], ["Frosinone", "https://i.imgur.com/x8qGMuV.png"]
#  ]

In [ ]:
# team_logos = \
# [["Arminia", "https://i.imgur.com/PKcXmoo.png"], ["Augsburg", "https://i.imgur.com/c9rRqYo.png"],
#  ["Bayern Munich", "https://i.imgur.com/rO1xVQh.png"], ["Bochum", "https://i.imgur.com/3LmZ5gX.png"],
#  ["Dortmund", "https://i.imgur.com/DyWKX0s.png"], ["Eint Frankfurt", "https://i.imgur.com/Fz4xh6m.png"],
#  ["Freiburg", "https://i.imgur.com/1JUxe7u.png"], ["Greuther Fürth", "https://i.imgur.com/ttJEUIb.png"],
#  ["Hertha BSC", "https://i.imgur.com/KAgTHoB.png"], ["Hoffenheim", "https://i.imgur.com/dooGO0m.png"],
#  ["Köln", "https://i.imgur.com/Gpz6jl7.png"], ["Leverkusen", "https://i.imgur.com/q3lV6o8.png"],
#  ["M'Gladbach", "https://i.imgur.com/K6aoRqW.png"], ["Mainz 05", "https://i.imgur.com/iOoxKkB.png"],
#  ["RB Leipzig", "https://i.imgur.com/CXfyKw4.png"], ["Stuttgart", "https://i.imgur.com/eQB0wsT.png"],
#  ["Union Berlin", "https://i.imgur.com/e5pjmHx.png"], ["Wolfsburg", "https://i.imgur.com/o2PP4ux.png"],
#  ["Schalke 04", "https://i.imgur.com/FfsE1Kk.png"], ["Werder Bremen", "https://i.imgur.com/XrAnCXM.png"],
#  ["Darmstadt 98", "https://i.imgur.com/CZbwcFI.png"], ["Heidenheim", "https://i.imgur.com/ks2nifY.png"]
#  ]

In [ ]:
# team_logos = \
# [["Barcelona", "https://i.imgur.com/1ZRO0Be.png"], ["Villarreal", "https://i.imgur.com/yQJvRBY.png"],
#  ["Valencia", "https://i.imgur.com/Erg2x2b.png"], ["Sevilla", "https://i.imgur.com/z3WXql2.png"],
#  ["Real Sociedad", "https://i.imgur.com/KdMxrGX.png"], ["Real Madrid", "https://i.imgur.com/pPD5ZAg.png"],
#  ["Rayo Vallecano", "https://i.imgur.com/cQ5Mv8I.png"], ["Osasuna", "https://i.imgur.com/6kAlaCy.png"],
#  ["Mallorca", "https://i.imgur.com/Qvdlcxy.png"], ["Levante", "https://i.imgur.com/8fZG4VA.png"],
#  ["Granada", "https://i.imgur.com/ce69kNa.png"], ["Getafe", "https://i.imgur.com/C4eV2p0.png"],
#  ["Espanyol", "https://i.imgur.com/RHTK1eL.png"], ["Elche", "https://i.imgur.com/S9bDHtO.png"],
#  ["Cádiz", "https://i.imgur.com/DztjVQE.png"], ["Celta Vigo", "https://i.imgur.com/kTsF0rV.png"],
#  ["Betis", "https://i.imgur.com/zyGTmAb.png"], ["Atlético Madrid", "https://i.imgur.com/x4c0wbO.png"],
#  ["Athletic Club", "https://i.imgur.com/qI6Uchm.png"], ["Alavés", "https://i.imgur.com/o4rSfuL.png"],
#  ["Almería", "https://i.imgur.com/kFyJaGO.png"], ["Girona", "https://i.imgur.com/0CwKO1D.png"],
#  ["Valladolid", "https://i.imgur.com/HCShRZI.png"], ["Las Palmas", "https://i.imgur.com/KAHlAuF.png"]
#  ]

In [ ]:
# team_names = team_names.merge(pd.DataFrame(team_logos, columns=["TEAM_NAME", "TEAM_LOGO_URL"]), on="TEAM_NAME")

In [ ]:
# ws_teams.rename({'team_name':'TEAM_NAME', 'team_id':'TEAM_WS_ID'}, axis=1, inplace=True)

In [ ]:
# team_names = team_names.merge(ws_teams, on="TEAM_NAME")

In [ ]:
upsert_to_snowflake("TEAMS", team_names, ['TEAM_FBREF_ID'])

In [ ]:
team_names = fetch_data(cur, 'SELECT * FROM TEAMS')

In [ ]:
team_names.drop(columns=['TEAM_LOGO_URL'], inplace=True)

In [ ]:
team_standard_stats = understat_table.copy()

In [ ]:
# team_standard_stats.reset_index(inplace=True)
# team_standard_stats.loc[team_standard_stats['TEAM_NAME'] == 'Luton', 'TEAM_NAME'] = 'Luton Town'
# team_standard_stats.set_index(['COMPETITION', 'SEASON', 'TEAM_NAME'], inplace=True)

In [ ]:
team_standard_stats['MATCHES_PLAYED'] = team_matches_played

In [ ]:
team_standard_stats.reset_index(inplace=True)

In [ ]:
team_standard_stats = team_standard_stats.merge(team_names, on='TEAM_NAME', how='left')

In [ ]:
team_standard_stats = team_standard_stats[['TEAM_FBREF_ID','SEASON','COMPETITION','MATCHES_PLAYED','TEAM_WINS', 
                                           'TEAM_DRAWS', 'TEAM_LOSSES', 'TEAM_PTS', 'TEAM_XPTS']]

In [ ]:
upsert_to_snowflake("TEAM_STANDARD_STATS", team_standard_stats, ['TEAM_FBREF_ID', 'COMPETITION','SEASON'])

In [ ]:
team_attacking_stats = understat_table.copy()

In [ ]:
team_attacking_stats = team_attacking_stats.reset_index().merge(team_names, on='TEAM_NAME', how='left')

In [ ]:
team_attacking_stats = team_attacking_stats[['TEAM_NAME','TEAM_FBREF_ID','COMPETITION','SEASON','NPxG','xG']]

In [ ]:
team_attacking_stats.rename(columns={'NPxG': 'NPXG', 'xG':'XG'}, inplace=True)

In [ ]:
team_attacking_stats.set_index(['COMPETITION', 'SEASON', 'TEAM_NAME'], inplace=True)

In [ ]:
team_attacking_stats['GOALS_SCORED'] = team_goals_scored
team_attacking_stats['SHOTS'] = team_shots
team_attacking_stats['SHOTS_ON_TARGET'] = team_shotsOT
team_attacking_stats['PASS_COMPLETED'] = team_pass_completed
team_attacking_stats['PASS_ATTEMPTED'] = team_pass_attempted
team_attacking_stats['TAKEONS_ATTEMPTED'] = team_takeons_attempted
team_attacking_stats['TAKEONS_COMPLETED'] = team_takeons_completed
team_attacking_stats['CROSSES_INTO_PA'] = team_crossesintoPA
team_attacking_stats['FOULS_AGAINST'] = team_fouls_against

In [ ]:
team_attacking_stats.reset_index(inplace=True)
team_attacking_stats.drop(columns=['TEAM_NAME'], inplace=True)

In [ ]:
upsert_to_snowflake("TEAM_ATTACKING_STATS", team_attacking_stats, ['TEAM_FBREF_ID', 'COMPETITION','SEASON'])

In [ ]:
team_defending_stats = understat_table.copy()

In [ ]:
team_defending_stats = team_defending_stats.reset_index().merge(team_names, on='TEAM_NAME', how='left')

In [ ]:
team_defending_stats = team_defending_stats[['TEAM_NAME','TEAM_FBREF_ID','COMPETITION','SEASON']]

In [ ]:
team_defending_stats.set_index(['COMPETITION', 'SEASON', 'TEAM_NAME'], inplace=True)

In [ ]:
team_defending_stats['GOALS_CONCEDED'] = team_goals_conceded
team_defending_stats['XG_AGAINST'] = understat_table['XG_AGAINST']
team_defending_stats['NPXG_AGAINST'] = understat_table['NPXG_AGAINST']
team_defending_stats['TACKLES'] = team_tackles
team_defending_stats['TACKLES_WON'] = team_tackles_won
team_defending_stats['FOULS_MADE'] = team_fouls_made
team_defending_stats['INTERCEPTIONS'] = team_interceptions
team_defending_stats['BLOCKED_SHOTS'] = team_blocks_shots
team_defending_stats['BLOCKED_PASSES'] = team_blocks_pass
team_defending_stats['CLEARANCES'] = team_clearances
team_defending_stats['PPDA'] = understat_table['PPDA']
team_defending_stats['OPP_PPDA'] = team_OPPDA
team_defending_stats['POSS_WON'] = teams_poss_won
team_defending_stats['FINAL_3RD_PASSES_AGAINST'] = team_final_3rd_passes_against
team_defending_stats['CLEAN_SHEETS'] = team_cleansheets
team_defending_stats['SHOTS_FACED'] = team_shots_faced

In [ ]:
team_defending_stats.reset_index(inplace=True)
team_defending_stats.drop(columns=['TEAM_NAME'], inplace=True)

In [ ]:
# sql_command = """
# ALTER TABLE GEGENSTATS.TABLES.TEAM_DEFENDING_STATS 
# ADD COLUMN SHOTS_FACED NUMBER(38,0);
# """

# cur.execute(sql_command)

In [ ]:
# team_shots_faced = team_shots_faced.reset_index()

In [ ]:
# for index, row in team_shots_faced.iterrows():
#     sql_command = """
#     UPDATE GEGENSTATS.TABLES.TEAM_DEFENDING_STATS
#     SET SHOTS_FACED = %s
#     WHERE TEAM_FBREF_ID = (SELECT TEAM_FBREF_ID FROM GEGENSTATS.TABLES.TEAMS WHERE TEAM_NAME = %s) 
#       AND COMPETITION = %s
#       AND SEASON = %s
#     """

#     values = (row['Sh'], row['TEAM_NAME'], row['COMPETITION'], row['SEASON'])

#     cur.execute(sql_command, values)

In [ ]:
upsert_to_snowflake("TEAM_DEFENDING_STATS", team_defending_stats, ['TEAM_FBREF_ID', 'COMPETITION','SEASON'])

In [ ]:
team_misc_stats = understat_table.copy()

In [ ]:
team_misc_stats = team_misc_stats.reset_index().merge(team_names, on='TEAM_NAME', how='left')

In [ ]:
team_misc_stats = team_misc_stats[['TEAM_NAME','TEAM_FBREF_ID','COMPETITION','SEASON']]

In [ ]:
team_misc_stats.set_index(['COMPETITION', 'SEASON', 'TEAM_NAME'], inplace=True)

In [ ]:
team_misc_stats['AERIALS_WON'] = team_aerials_won
team_misc_stats['AERIALS_LOST'] = team_aerials_lost

In [ ]:
team_misc_stats.reset_index(inplace=True)
team_misc_stats.drop(columns=['TEAM_NAME'], inplace=True)

In [ ]:
upsert_to_snowflake("TEAM_MISC_STATS", team_misc_stats, ['TEAM_FBREF_ID', 'COMPETITION','SEASON'])

## Competition, Season and Match Data

In [ ]:
df_competitions = fbref_schedule.reset_index()[['COMPETITION', 'SEASON']].drop_duplicates().copy()

In [ ]:
df_competitions['COMPETITION_NAME'] = ''
df_competitions['COMPETITION_ACRONYM'] = ''

In [ ]:
df_competitions.loc[df_competitions['COMPETITION'] == 'ENG-Premier League', 'COMPETITION_NAME'] = 'English Premier League'
df_competitions.loc[df_competitions['COMPETITION'] == 'ENG-Premier League', 'COMPETITION_ACRONYM'] = 'EPL'

In [ ]:
df_competitions

In [ ]:
# upsert_to_snowflake("COMPETITIONS", df_competitions, ['COMPETITION','SEASON'])

In [ ]:
df_seasons = df_competitions[['SEASON']].drop_duplicates().copy()

In [ ]:
df_seasons['SEASON_NAME'] = ''
df_seasons['START_YEAR'] = 0
df_seasons['END_YEAR'] = 0

In [ ]:
# df_seasons.loc[df_seasons['SEASON'] == 2122, 'SEASON_NAME'] = '2021-2022'
# df_seasons.loc[df_seasons['SEASON'] == 2122, 'START_YEAR'] = 2021
# df_seasons.loc[df_seasons['SEASON'] == 2122, 'END_YEAR'] = 2022
df_seasons.loc[df_seasons['SEASON'] == 2324, 'SEASON_NAME'] = '2023-2024'
df_seasons.loc[df_seasons['SEASON'] == 2324, 'START_YEAR'] = 2023
df_seasons.loc[df_seasons['SEASON'] == 2324, 'END_YEAR'] = 2024

In [ ]:
df_seasons

In [ ]:
upsert_to_snowflake("SEASONS", df_seasons, ['SEASON'])

In [ ]:
df_stadiums = fbref_schedule.reset_index()[['home_team', 'SEASON',
            'venue']].drop_duplicates().rename(columns={'home_team': 'TEAM_NAME', 'venue': 'STADIUM'})

In [ ]:
df_stadiums = df_stadiums.merge(team_names, on='TEAM_NAME')[['STADIUM','SEASON','TEAM_FBREF_ID']]

In [ ]:
upsert_to_snowflake("STADIUMS", df_stadiums, ['TEAM_FBREF_ID','STADIUM', 'SEASON'])

In [ ]:
df_matches = fbref_schedule.reset_index()[['game_id', 'date', 'time', 'home_team', 'away_team', 'COMPETITION', 'SEASON', 'venue',
                                           'week', 'day', 'score', 'home_xg', 'away_xg', 'attendance', 'referee', 'ws_game_id']]

In [ ]:
df_matches.shape

In [ ]:
df_matches = df_matches.merge(team_names[['TEAM_FBREF_ID','TEAM_NAME']].rename(columns={'TEAM_FBREF_ID': 'home_team_id', 
                                                         'TEAM_NAME': 'home_team'}), on='home_team', how='left')

In [ ]:
df_matches = df_matches.merge(team_names[['TEAM_FBREF_ID','TEAM_NAME']].rename(columns={'TEAM_FBREF_ID': 'away_team_id', 
                                                         'TEAM_NAME': 'away_team'}), on='away_team', how='left')

In [ ]:
# Combine df_matches date and time to a dt.datatime configuration
df_matches['date'] = df_matches['date'].astype(str)
df_matches['time'] = df_matches['time'].astype(str)
df_matches['date_time'] = df_matches['date'] + ' ' + df_matches['time']
df_matches['date_time'] = pd.to_datetime(df_matches['date_time'])

In [ ]:
# Split score in df_matches by "-" to home_team_score and away_team_score 
df_matches[['home_team_score', 'away_team_score']] = df_matches['score'].str.split('–', expand=True)

In [ ]:
df_matches.rename(columns={'game_id': 'MATCH_ID', 'date_time': 'DATE_TIME', 'home_team_id': 'HOME_TEAM_ID', 'away_team_id': 'AWAY_TEAM_ID',
                           'venue': 'STADIUM', 'week': 'GAMEWEEK', 'day': 'DAY','home_team_score': 'HOME_TEAM_SCORE', 
                           'away_team_score':'AWAY_TEAM_SCORE', 'home_xg':'HOME_TEAM_XG','away_xg':'AWAY_TEAM_XG',
                           'attendance':'ATTENDANCE', 'referee': 'REFEREE', 'ws_game_id':'WS_MATCH_ID'}, inplace=True)

In [ ]:
df_matches.drop(columns=['date', 'time', 'home_team', 'away_team', 'score'], inplace=True)

In [ ]:
df_matches['DATE_TIME'] = df_matches['DATE_TIME'].astype('string')

In [ ]:
upsert_to_snowflake("MATCHES", df_matches, ['MATCH_ID'])

### Time to deal with event data

In [ ]:
game_ids = [1746344]

In [ ]:
ws_event_spadl = ws.read_events(output_fmt='spadl', match_id=game_ids)

In [ ]:
# for i, val in enumerate(game_ids):
#     if i == 0:
#         ws_event_spadl = ws.read_events(output_fmt='spadl', match_id=[val])
#     else:
#         try:
#             ws_event_spadl = pd.concat([ws_event_spadl, ws.read_events(output_fmt='spadl', match_id=[val])])
#         except:
#             print(i, val)

In [ ]:
len(ws_event_spadl['game_id'].unique())

In [ ]:
ws_event_spadl.to_csv('ws_event_spadl_hold.csv')
# ws_event_spadl = pd.read_csv('ws_event_spadl_hold.csv', index_col=0)

In [ ]:
sorted(ws_teams['TEAM_NAME']) == sorted(ws_event_spadl['team'].unique())

In [ ]:
ws_event_spadl.loc[ws_event_spadl['team'] == 'Atletico', 'team'] = 'Atlético Madrid'
ws_event_spadl.loc[ws_event_spadl['team'] == 'Deportivo Alaves', 'team'] = 'Alavés'
ws_event_spadl.loc[ws_event_spadl['team'] == 'Cadiz', 'team'] = 'Cádiz'
ws_event_spadl.loc[ws_event_spadl['team'] == 'Real Betis', 'team'] = 'Betis'
ws_event_spadl.loc[ws_event_spadl['team'] == 'Real Valladolid', 'team'] = 'Valladolid'
ws_event_spadl.loc[ws_event_spadl['team'] == 'Almeria', 'team'] = 'Almería'

ws_event_spadl.loc[ws_event_spadl['team'] == 'Arminia Bielefeld', 'team'] = 'Arminia'
ws_event_spadl.loc[ws_event_spadl['team'] == 'Bayern', 'team'] = 'Bayern Munich'
ws_event_spadl.loc[ws_event_spadl['team'] == 'Borussia Dortmund', 'team'] = 'Dortmund'
ws_event_spadl.loc[ws_event_spadl['team'] == 'Borussia M.Gladbach', 'team'] = "M'Gladbach"
ws_event_spadl.loc[ws_event_spadl['team'] == 'Eintracht Frankfurt', 'team'] = "Eint Frankfurt"
ws_event_spadl.loc[ws_event_spadl['team'] == 'FC Koln', 'team'] = 'Köln'
ws_event_spadl.loc[ws_event_spadl['team'] == 'Mainz', 'team'] = "Mainz 05"
ws_event_spadl.loc[ws_event_spadl['team'] == 'RBL', 'team'] = "RB Leipzig"
ws_event_spadl.loc[ws_event_spadl['team'] == 'Greuther Fuerth', 'team'] = "Greuther Fürth"
ws_event_spadl.loc[ws_event_spadl['team'] == 'Hertha Berlin', 'team'] = "Hertha BSC"
ws_event_spadl.loc[ws_event_spadl['team'] == 'Schalke', 'team'] = 'Schalke 04'
ws_event_spadl.loc[ws_event_spadl['team'] == 'Darmstadt', 'team'] = 'Darmstadt 98'
ws_event_spadl.loc[ws_event_spadl['team'] == 'FC Heidenheim', 'team'] = 'Heidenheim'

ws_event_spadl.loc[ws_event_spadl['team'] == 'Verona', 'team'] = 'Hellas Verona'

ws_event_spadl.loc[ws_event_spadl['team'] == 'PSG', 'team'] = "Paris S-G"
ws_event_spadl.loc[ws_event_spadl['team'] == 'Saint-Etienne', 'team'] = "Saint-Étienne"
ws_event_spadl.loc[ws_event_spadl['team'] == 'AC Ajaccio', 'team'] = "Ajaccio"

ws_event_spadl.loc[ws_event_spadl['team'] == 'Manchester United', 'team'] = 'Manchester Utd'
ws_event_spadl.loc[ws_event_spadl['team'] == 'Newcastle United', 'team'] = 'Newcastle Utd'
ws_event_spadl.loc[ws_event_spadl['team'] == 'Newcastle', 'team'] = 'Newcastle Utd'
ws_event_spadl.loc[ws_event_spadl['team'] == 'Tottenham Hotspur', 'team'] = 'Tottenham'
ws_event_spadl.loc[ws_event_spadl['team'] == 'Wolverhampton Wanderers', 'team'] = 'Wolves'
ws_event_spadl.loc[ws_event_spadl['team'] == 'Wolverhampton', 'team'] = 'Wolves'
ws_event_spadl.loc[ws_event_spadl['team'] == 'Nottingham Forest', 'team'] = "Nott'ham Forest"
ws_event_spadl.loc[ws_event_spadl['team'] == 'Leicester', 'team'] = "Leicester City"
ws_event_spadl.loc[ws_event_spadl['team'] == 'Leeds', 'team'] = "Leeds United"
ws_event_spadl.loc[ws_event_spadl['team'] == 'Sheffield United', 'team'] = "Sheffield Utd"
ws_event_spadl.loc[ws_event_spadl['team'] == 'Sheff Utd', 'team'] = "Sheffield Utd"
ws_event_spadl.loc[ws_event_spadl['team'] == 'Luton', 'team'] = "Luton Town"
ws_event_spadl.loc[ws_event_spadl['team'] == 'Norwich', 'team'] = "Norwich City"
ws_event_spadl.loc[ws_event_spadl['team'] == 'Man City', 'team'] = "Manchester City"
ws_event_spadl.loc[ws_event_spadl['team'] == 'Man Utd', 'team'] = "Manchester Utd"

In [ ]:
curr_team_names = fetch_data(cur, 'SELECT * FROM TEAMS')

In [ ]:
ws_event_spadl.shape

In [ ]:
ws_event_spadl = ws_event_spadl.merge(spadl.actiontypes_df(), how="left").merge(spadl.bodyparts_df(), how="left")

In [ ]:
ws_event_spadl.rename({'game_id':'WS_MATCH_ID', 'original_event_id':'ORIGINAL_EVENT_ID','period_id':'PERIOD_ID',
                       'time_seconds':'TIME_SECONDS', 'team_id':'TEAM_WS_ID', 'player_id':'PLAYER_WS_ID', 'start_x':'START_X',
                        'end_x':'END_X', 'start_y':'START_Y', 'end_y':'END_Y','type_name':'TYPE_NAME',
                        'bodypart_name':'BODYPART_NAME','result_id':'RESULT_ID', 'action_id':'ACTION_ID'}, axis=1, inplace=True)

In [ ]:
ws_event_spadl = ws_event_spadl.merge(df_matches[['MATCH_ID', 'WS_MATCH_ID']], on='WS_MATCH_ID')

In [ ]:
ws_event_spadl = ws_event_spadl.merge(curr_team_names[['TEAM_FBREF_ID','TEAM_WS_ID']], on='TEAM_WS_ID')

In [ ]:
ws_event_spadl.rename({'TEAM_FBREF_ID':'team_id','START_X':'start_x','START_Y':'start_y','END_X':'end_x','END_Y':'end_y'}, axis=1,
                      inplace=True)

In [ ]:
df_matches_left =df_matches[df_matches['MATCH_ID'].isin(ws_event_spadl['MATCH_ID'].unique())]

In [ ]:
# df_matches_left = df_matches[df_matches['MATCH_ID'].isin(matches_to_scarpe)]

In [ ]:
for index, row in df_matches_left.iterrows():
    inds_to_replace = ws_event_spadl[ws_event_spadl['MATCH_ID'] == row['MATCH_ID']].index
    ws_event_spadl.loc[inds_to_replace] = spadl.play_left_to_right(ws_event_spadl.loc[inds_to_replace], row['HOME_TEAM_ID'])

In [ ]:
ws_event_spadl['NEW_TIME_SECONDS'] = ws_event_spadl['TIME_SECONDS']

In [ ]:
ws_event_spadl_period1 = ws_event_spadl[ws_event_spadl['PERIOD_ID'] == 1]
ws_event_spadl_period2 = ws_event_spadl[ws_event_spadl['PERIOD_ID'] == 2]

In [ ]:
start_time_2nd_half = ws_event_spadl_period2.groupby(['MATCH_ID', 'PERIOD_ID'])['NEW_TIME_SECONDS'].min().reset_index()

In [ ]:
for index, row in tqdm(start_time_2nd_half.iterrows()):
    time_to_remove = row['NEW_TIME_SECONDS']
    ws_event_spadl.loc[(ws_event_spadl['MATCH_ID'] == row['MATCH_ID']) & (ws_event_spadl['PERIOD_ID'] == 2), 'NEW_TIME_SECONDS'] -= time_to_remove

In [ ]:
ws_event_spadl.rename({'start_x':'START_X','end_x':'END_X','start_y':'START_Y','end_y':'END_Y',
                       'team_id':'TEAM_FBREF_ID'}, axis=1, inplace=True)

In [ ]:
ws_event_spadl.shape

In [ ]:
events_drop_inds = ws_event_spadl[ws_event_spadl[['WS_MATCH_ID', 'PLAYER_WS_ID','ACTION_ID']].duplicated()].index

In [ ]:
len(events_drop_inds)

In [ ]:
ws_event_spadl = ws_event_spadl.drop(events_drop_inds)

In [ ]:
upsert_to_snowflake("EVENTS_SPADL", 
                    ws_event_spadl.drop(['type_id', 'player', 'team', 'WS_MATCH_ID', 'TEAM_WS_ID','bodypart_id'],axis=1), 
                    ['MATCH_ID','ACTION_ID'])

In [ ]:
# cur.execute('SELECT * FROM EVENTS_SPADL')
# events_rows = cur.fetchall()
# column_names = [desc[0] for desc in cur.description]
# ws_event_spadl = pd.DataFrame(events_rows, columns=column_names)

### Update the "Players" table in Snowflake

In [ ]:
## Sort out "Players" "Table
ws_players.rename({'team_id':'TEAM_WS_ID', 'game_id':'WS_MATCH_ID'}, axis=1, inplace=True)

In [ ]:
ws_players = ws_players.merge(curr_team_names, on="TEAM_WS_ID", how="left")

In [ ]:
ws_players = ws_players.sort_values(by=["WS_MATCH_ID","TEAM_NAME","jersey_number"])

In [ ]:
fbref_lineups.shape

In [ ]:
fbref_lineups = fbref_lineups.reset_index().merge(fbref_schedule.reset_index()[['game', 'game_id',
                                                                                'ws_game_id']], how='inner', on='game')

In [ ]:
fbref_lineups = fbref_lineups.rename({'ws_game_id':'WS_MATCH_ID'}, axis=1)

In [ ]:
fbref_lineups.shape

In [ ]:
fbref_lineups = fbref_lineups.merge(curr_team_names[['TEAM_NAME','TEAM_FBREF_ID','TEAM_WS_ID']], on='TEAM_NAME')

In [ ]:
ws_players.rename({'player_id':'WS_player_id', 'player_name':'WS_player_name','minutes_played':'WS_minutes_played',
                   'starting_position':'WS_starting_position', 'is_starter':'WS_is_starter'}, axis=1, inplace=True)

In [ ]:
fbref_lineups.rename({'minutes_played':'FBREF_minutes_played', 'game_id':'MATCH_ID','position':'FBREF_position',
                      'is_starter':'FBREF_is_starter'}, axis=1, inplace=True)

In [ ]:
fbref_lineups = fbref_lineups.merge(ws_players, on=["TEAM_FBREF_ID", "TEAM_NAME", "WS_MATCH_ID",
                                                    "TEAM_WS_ID", "jersey_number"], how="left")

In [ ]:
df_players = fbref_lineups[['WS_player_id', 'player', 'WS_player_name']].drop_duplicates()

In [ ]:
df_players = df_players.dropna(subset=['WS_player_id'])

In [ ]:
df_players.rename({'WS_player_id':'PLAYER_WS_ID', 'player':'PLAYER_FBREF_NAME', 'WS_player_name':'PLAYER_WS_NAME'}, 
                  axis=1, inplace=True)

In [ ]:
df_players[df_players['PLAYER_WS_ID'].duplicated()]

In [ ]:
df_players = df_players.replace('Ronald Araujo', 'Ronald Araújo')
df_players = df_players.replace('Íñigo Martínez', 'Iñigo Martínez')

In [ ]:
df_players = df_players.replace('Lucas Michel', 'Lucas Michal')
df_players = df_players.replace('Théo Ramousse', 'Théo Borne')
df_players = df_players.replace('Joel Mugisha', 'Joel Mvuka')

In [ ]:
df_players = df_players[(df_players['PLAYER_FBREF_NAME'] != 'Destiny Egharevba') | (df_players['PLAYER_WS_ID'] != 428502)]

In [ ]:
df_players = df_players[(df_players['PLAYER_FBREF_NAME'] != 'Sacha Delaye') | (df_players['PLAYER_WS_NAME'] != 'Enzo Tchato')]

In [ ]:
df_players = df_players.replace('Malcolm Barcola', 'Bradley Barcola')

In [ ]:
df_players = df_players[['PLAYER_WS_ID', 'PLAYER_FBREF_NAME', 'PLAYER_WS_NAME']].drop_duplicates()

In [ ]:
# df_players[df_players['PLAYER_WS_ID']==426915]

In [ ]:
## Find all instances of "Min-Jae Kim" in df_players

# df_players[df_players['PLAYER_FBREF_NAME'].str.contains('Luca Ravanelli')]

# # Replace it with "Kim Min-Jae"

# df_players.loc[df_players['PLAYER_WS_ID'] == 426915, 'PLAYER_FBREF_NAME'] = 'Felix Afena-Gyan'

In [ ]:
upsert_to_snowflake("PLAYERS", df_players, ['PLAYER_WS_ID'])

Player-match combos now

In [ ]:
df_player_match = fbref_lineups[['WS_player_id', 'player','MATCH_ID', 'TEAM_FBREF_ID', 'WS_starting_position', 
                                 'FBREF_position','jersey_number','FBREF_minutes_played','FBREF_is_starter','WS_is_starter',
                                 'WS_minutes_played']]

In [ ]:
df_player_match = df_player_match.rename({'WS_player_id':'PLAYER_WS_ID', 'player':'PLAYER_FBREF_NAME', 'WS_starting_position':'STARTING_POSITION_WS',
                        'FBREF_position':'POSITION_FBREF', 'jersey_number':'JERSEY_NUMBER', 'FBREF_minutes_played':'MINUTES_PLAYED_FBREF',
                        'FBREF_is_starter':'IS_STARTER_FBREF','WS_is_starter':'IS_STARTER_WS','WS_minutes_played':'MINUTES_PLAYED_WS'}, axis=1)

In [ ]:
inds_to_drop = df_player_match[df_player_match[['PLAYER_WS_ID', 'MATCH_ID']].duplicated()].index

In [ ]:
inds_to_drop

In [ ]:
df_player_match = df_player_match.drop(inds_to_drop)

In [ ]:
df_player_match = df_player_match.dropna(subset=['PLAYER_WS_ID'])

In [ ]:
df_player_match = df_player_match[~((df_player_match['MATCH_ID'] == 'd6d3de2f') & 
                                    (df_player_match['PLAYER_WS_ID'] == 502424))]

In [ ]:
df_player_match = df_player_match[~((df_player_match['MATCH_ID'] == '1a0a12f3') & 
                                    (df_player_match['PLAYER_WS_ID'] == 135328))]

In [ ]:
upsert_to_snowflake("PLAYER_MATCH", df_player_match, ['PLAYER_WS_ID', 'MATCH_ID'])

### Add Shot Event Data

In [ ]:
no_shot_match = []

In [ ]:
matches_to_scarpe1 = df_matches[df_matches['MATCH_ID'].isin(ws_event_spadl['MATCH_ID'].unique())]['MATCH_ID']

In [ ]:
len(matches_to_scarpe1)

In [ ]:
for i, val in tqdm(enumerate(matches_to_scarpe1)):
    if i == 0:
        fbref_shot_events = fbref.read_shot_events(match_id=val)
    else:
        try:
            fbref_shot_events = pd.concat([fbref_shot_events, fbref.read_shot_events(match_id=val)])
        except:
            no_shot_match.append(val)

In [ ]:
no_shot_match

In [ ]:
fbref_shot_events = fbref_shot_events.rename_axis(index={'season': 'SEASON', 'league': 'COMPETITION'})

In [ ]:
fbref_shot_events = make_season_integer(fbref_shot_events)

In [ ]:
ws_event_spadl['TIME_SECONDS'] = ws_event_spadl['TIME_SECONDS'].astype(int)
ws_event_spadl['PLAYER_WS_ID'] = ws_event_spadl['PLAYER_WS_ID'].astype(int)

In [ ]:
fbref_shot_events.shape

In [ ]:
ws_shots = ws_event_spadl[(ws_event_spadl['TYPE_NAME'] == 'shot') | (ws_event_spadl['TYPE_NAME'] == 'shot_freekick') | \
               (ws_event_spadl['TYPE_NAME'] == 'shot_penalty')]

In [ ]:
ws_shots.shape

In [ ]:
fbref_shot_events.reset_index(inplace=True)

In [ ]:
fbref_shot_events.columns = fbref_shot_events.columns.get_level_values(0) + fbref_shot_events.columns.get_level_values(1)

In [ ]:
fbref_shot_events.to_csv('fbref_shot_events.csv')
# fbref_shot_events = pd.read_csv('fbref_shot_events.csv', index_col=0)

In [ ]:
fbref_shot_events = fbref_shot_events.merge(fbref_schedule[['COMPETITION','SEASON','game','game_id','ws_game_id']], 
                        on=['COMPETITION','SEASON','game'])

In [ ]:
fbref_shot_events.rename({'player':'PLAYER_FBREF_NAME', 'game_id':'MATCH_ID'}, axis=1, inplace=True)

In [ ]:
fbref_shot_events['PLAYER_FBREF_NAME'] = fbref_shot_events['PLAYER_FBREF_NAME'].str.replace(' \(pen\)', '', regex=True)

In [ ]:
fbref_shot_events.loc[fbref_shot_events['PLAYER_FBREF_NAME'] == 'Loum Ndiaye', 'PLAYER_FBREF_NAME'] = "Mamadou N'Diaye"

In [ ]:
len(df_player_match['MATCH_ID'].unique())

In [ ]:
df_player_match = df_player_match[~((df_player_match['MATCH_ID'] == 'd6d3de2f') & 
                                    (df_player_match['PLAYER_WS_ID'] == 502424))]
df_player_match = df_player_match[~((df_player_match['MATCH_ID'] == '1a0a12f3') & 
                                    (df_player_match['PLAYER_WS_ID'] == 135328))]

In [ ]:
fbref_shot_events = fbref_shot_events.merge(df_player_match[['PLAYER_WS_ID', 'PLAYER_FBREF_NAME', 'MATCH_ID']], 
                        on=['PLAYER_FBREF_NAME', 'MATCH_ID'], how="inner")

In [ ]:
fbref_shot_events = fbref_shot_events[(fbref_shot_events['MATCH_ID'] != '7e4bbae4') | 
                  (fbref_shot_events['PLAYER_FBREF_NAME'] != 'Borja Iglesias')]

In [ ]:
fbref_shot_events = fbref_shot_events[(fbref_shot_events['MATCH_ID'] != '7e4bbae4') | 
                  (fbref_shot_events['PLAYER_FBREF_NAME'] != 'Roberto González')]

In [ ]:
fbref_shot_events = fbref_shot_events[(fbref_shot_events['MATCH_ID'] != '82b8a000') |  (fbref_shot_events['minute'] != '81')]

In [ ]:
fbref_shot_events = fbref_shot_events[(fbref_shot_events['MATCH_ID'] != '9bd36df7') |  
                                      (fbref_shot_events['PLAYER_FBREF_NAME'] != 'Saîf-Eddine Khaoui') |
                                      (fbref_shot_events['minute'] != '38')]

In [ ]:
fbref_shot_events = fbref_shot_events[(fbref_shot_events['MATCH_ID'] != 'f7d8a40e') |  
                                      (fbref_shot_events['PLAYER_FBREF_NAME'] != 'Matteo Prati') |
                                      (fbref_shot_events['minute'] != '77')]

In [ ]:
# fbref_shot_events = fbref_shot_events.merge(df_players, on='PLAYER_FBREF_NAME', how="left")

In [ ]:
# ws_shots = ws_event_spadl[(ws_event_spadl['TYPE_NAME'] == 'shot') | (ws_event_spadl['TYPE_NAME'] == 'shot_freekick') | \
#                (ws_event_spadl['TYPE_NAME'] == 'shot_penalty')]

In [ ]:
ws_shots['ORIGINAL_EVENT_ID'] = ws_shots['ORIGINAL_EVENT_ID'].astype(float)

In [ ]:
ws_shots = ws_shots.drop_duplicates()

In [ ]:
ws_shots = ws_shots[(ws_shots['MATCH_ID'] != '7e4bbae4')| 
        (ws_shots['player'] != 'Rober')]

In [ ]:
ws_shots = ws_shots[(ws_shots['MATCH_ID'] != '7e4bbae4')| 
        (ws_shots['player'] != 'Borja Iglesias')]

In [ ]:
ws_shots = ws_shots[(ws_shots['MATCH_ID'] != '47134469')| 
        (ws_shots['player'] != 'Ángel Alarcón')]

In [ ]:
ws_shots = ws_shots[(ws_shots['MATCH_ID'] != 'c34bbc21')]

In [ ]:
# ws_dropped_shots = ws_shots[ws_shots['PLAYER_WS_ID'] == 444304]
# ws_shots = ws_shots[ws_shots['PLAYER_WS_ID'] != 444304]

In [ ]:
## Bamba from Lorient's shots to be dropped
ws_dropped_shots = ws_shots[ws_shots['PLAYER_WS_ID'] == 515493]
ws_shots = ws_shots[ws_shots['PLAYER_WS_ID'] != 515493]

In [ ]:
ws_shots = ws_shots[ws_shots['PLAYER_WS_ID'] != 509340]
ws_shots = ws_shots[ws_shots['PLAYER_WS_ID'] != 511120]
ws_shots = ws_shots[ws_shots['PLAYER_WS_ID'] != 492362]

In [ ]:
fbref_shot_events.shape

In [ ]:
ws_shots.shape

In [ ]:
## Rogue Joel Veltman shot should be dropped
ws_shots = ws_shots.drop(ws_shots[(ws_shots['PLAYER_WS_ID'] == 110290) & 
                                  (ws_shots['MATCH_ID'] == '36be5cee')].index)

In [ ]:
## Rogue Negredo shot to drop
ws_shots = ws_shots[(ws_shots['MATCH_ID'] != 'c409c152') | (ws_shots['PLAYER_WS_ID'] != 23757) | (ws_shots['ACTION_ID'] != 149)]

In [ ]:
ws_shots = ws_shots[(ws_shots['MATCH_ID'] != '84765ee6') | (ws_shots['ACTION_ID'] != 117)]

In [ ]:
ws_shots = ws_shots[(ws_shots['MATCH_ID'] != 'b63b2da7') | (ws_shots['WS_MATCH_ID'] != 1727122)]

In [ ]:
## Rogue Negredo shot to drop
ws_shots = ws_shots[(ws_shots['MATCH_ID'] != 'de22618d') | (ws_shots['PLAYER_WS_ID'] != 286828) | (ws_shots['ACTION_ID'] != 1716)]

In [ ]:
## Rogue Rodrigo Riquelme shot to drop
ws_shots = ws_shots[(ws_shots['MATCH_ID'] != 'bccce279') | (ws_shots['PLAYER_WS_ID'] != 382631) | (ws_shots['ACTION_ID'] != 958)]

In [ ]:
## Rogue Rodrigo Riquelme shot to drop
ws_shots = ws_shots[(ws_shots['MATCH_ID'] != '55996670') | (ws_shots['PLAYER_WS_ID'] != 403649) | (ws_shots['ACTION_ID'] != 1229)]

In [ ]:
relevant_inds = fbref_shot_events.drop(['ws_game_id', 'PLAYER_WS_ID'], axis=1).drop_duplicates().index
relevant_inds

In [ ]:
fbref_shot_events = fbref_shot_events.loc[list(relevant_inds)]

In [ ]:
# Get the indices of the last 5 rows where MATCH_ID == '4c975b7e'
indices = fbref_shot_events[fbref_shot_events['MATCH_ID'] == '4c975b7e'].index[-5:]
fbref_shot_events = fbref_shot_events.drop(indices)

In [ ]:
ws_shots.shape

In [ ]:
for game in ws_shots['MATCH_ID'].unique():
    if len(ws_shots.loc[ws_shots.MATCH_ID == game]) != len(fbref_shot_events.loc[fbref_shot_events.MATCH_ID == game]):
        print(game)

In [ ]:
# fbref_shot_events[['new_minute', 'added_minute']] = fbref_shot_events['minute'].apply(
#     lambda x: pd.Series(str(x).split('+') if '+' in str(x) else [x, 0])
# )

In [ ]:
# fbref_shot_events['new_minute'] = fbref_shot_events['new_minute'].astype(int)
# fbref_shot_events['added_minute'] = fbref_shot_events['added_minute'].astype(int)

In [ ]:
# fbref_shot_events = fbref_shot_events.sort_values(by=['MATCH_ID', 'new_minute', 'added_minute', 'PLAYER_WS_ID']).reset_index(drop=True)

In [ ]:
ws_shots = ws_shots.sort_values(by=['MATCH_ID', 'PERIOD_ID','ACTION_ID']).reset_index(drop=True)

In [ ]:
ws_shots['xG'] = np.nan
ws_shots['PSxG'] = np.nan
ws_shots['outcome'] = np.nan
ws_shots['distance'] = np.nan
ws_shots['body_part'] = np.nan
ws_shots['notes'] = np.nan
ws_shots['SCA 1player'] = np.nan
ws_shots['SCA 1event'] = np.nan
ws_shots['SCA 2player'] = np.nan
ws_shots['SCA 2event'] = np.nan
ws_shots['FBREF_PLAYER_WS_ID'] = np.nan
ws_shots['PLAYER_WS_NAME'] = np.nan

In [ ]:
fbref_shot_events.rename({'PLAYER_WS_ID':'FBREF_PLAYER_WS_ID'}, axis=1, inplace=True)

In [ ]:
fbref_shot_events = fbref_shot_events.merge(df_players.rename({'PLAYER_WS_ID':'FBREF_PLAYER_WS_ID'}, axis=1), 
                        on=['FBREF_PLAYER_WS_ID', 'PLAYER_FBREF_NAME'], how="left")

In [ ]:
cols_of_interest = ['xG', 'PSxG', 'outcome', 'distance', 'body_part', 'notes', 'SCA 1player', 'SCA 1event', 
                    'SCA 2player', 'SCA 2event', 'FBREF_PLAYER_WS_ID', 'PLAYER_WS_NAME']
for game in ws_shots['MATCH_ID'].unique():
    df_hold = fbref_shot_events[fbref_shot_events['MATCH_ID'] == game]
    ind_list = ws_shots[ws_shots['MATCH_ID'] == game].index
    ws_shots.loc[ind_list, cols_of_interest] = np.array(df_hold[cols_of_interest])

In [ ]:
ws_shots['FBREF_PLAYER_WS_ID'] = ws_shots['FBREF_PLAYER_WS_ID'].astype(int)

In [ ]:
mismatched_inds = ws_shots[ws_shots['PLAYER_WS_ID'] != ws_shots['FBREF_PLAYER_WS_ID']].index

In [ ]:
mismatched_inds

In [ ]:
# ONLY for the indecies with mismatched_inds, cahnge "FBREF_PLAYER_WS_ID" to teh value in "PLAYER_WS_ID" if "player" column's value is equal to "PLAYER_WS_NAME" column's value
mask = ws_shots.loc[mismatched_inds, 'player'] == ws_shots.loc[mismatched_inds, 'PLAYER_WS_NAME']
ws_shots.loc[mismatched_inds[mask], 'FBREF_PLAYER_WS_ID'] = ws_shots.loc[mismatched_inds[mask], 'PLAYER_WS_ID']

In [ ]:
mismatched_inds = ws_shots[ws_shots['PLAYER_WS_ID'] != ws_shots['FBREF_PLAYER_WS_ID']].index

In [ ]:
mask = (ws_shots.loc[mismatched_inds, 'player'] == 'Vítinha')
ws_shots.loc[mismatched_inds[mask], 'FBREF_PLAYER_WS_ID'] = ws_shots.loc[mismatched_inds[mask], 'PLAYER_WS_ID']

In [ ]:
for i in range(len(mismatched_inds) // 2):
    val1 = ws_shots.loc[mismatched_inds, cols_of_interest].iloc[2*i]
    val2 = ws_shots.loc[mismatched_inds, cols_of_interest].iloc[(2*i)+1]

    ws_shots.loc[val1.name, cols_of_interest] = np.array(val2)
    ws_shots.loc[val2.name, cols_of_interest] = np.array(val1)

In [ ]:
#### Remove this part later becasue it's just for Yarmolyuik

df_shots = ws_shots[['MATCH_ID','ACTION_ID','xG', 'PSxG', 'outcome', 'distance', 
          'body_part', 'notes', 'SCA 1player', 'SCA 1event', 'SCA 2player', 'SCA 2event']]

# ws_dropped_shots[[cols_of_interest]] = np.nan

# rows_to_add = [
#     [0.04,np.nan,'Blocked',16,'Right Foot',np.nan,np.nan,np.nan,np.nan,np.nan,444304,'Yehor Yarmoliuk'],
# [0.03,0.12,'Saved',27,'Right Foot',np.nan,np.nan,np.nan,np.nan,np.nan,444304,'Yehor Yarmoliuk'],
# [0.05,np.nan,'Off Target',21,'Right Foot',np.nan,np.nan,np.nan,np.nan,np.nan,444304,'Yehor Yarmoliuk'],
# [0.49,0.25,'Saved',11,'Right Foot',np.nan,np.nan,np.nan,np.nan,np.nan,444304,'Yehor Yarmoliuk'],
# [0.09,np.nan,'Off Target',8,'Right Foot',np.nan,np.nan,np.nan,np.nan,np.nan,444304,'Yehor Yarmoliuk'],
# [0.02,0.02,'Saved',19,'Right Foot',np.nan,np.nan,np.nan,np.nan,np.nan,444304,'Yehor Yarmoliuk']
# ]

# ws_dropped_shots[cols_of_interest] = np.array(rows_to_add)

In [ ]:
# df_shots = pd.concat([df_shots, ws_dropped_shots[['MATCH_ID','ACTION_ID','xG', 'PSxG', 'outcome', 'distance', 
#           'body_part', 'notes', 'SCA 1player', 'SCA 1event', 'SCA 2player', 'SCA 2event']]]).sort_index()

In [ ]:
# df_shots.replace({"Jarrell Quansah": "Jarell Quansah"}, inplace=True)
# df_shots.replace({"Omotayo Adaramola": "Tayo Adaramola"}, inplace=True)

In [ ]:
df_shots.rename({'xG':'XG', 'PSxG':'PSXG', 'outcome':'OUTCOME', 'distance':'DISTANCE', 'body_part':'BODY_PART',
                 'notes':'NOTES', 'SCA 1player':'SCA_1_PLAYER_FBREF_NAME', 'SCA 1event':'SCA_1_PLAYER_FBREF_EVENT',
                 'SCA 2player':'SCA_2_PLAYER_FBREF_NAME', 'SCA 2event':'SCA_2_PLAYER_FBREF_EVENT'}, axis=1, inplace=True)

In [ ]:
# df_players.rename({'PLAYER_FBREF_NAME':'SCA_1_PLAYER_FBREF_NAME'}, axis=1, inplace=True)
# df_players.rename({'PLAYER_WS_ID':'SCA_1_PLAYER_FBREF_ID'}, axis=1, inplace=True)

In [ ]:
df_player_match.rename({'PLAYER_FBREF_NAME':'SCA_1_PLAYER_FBREF_NAME'}, axis=1, inplace=True)
df_player_match.rename({'PLAYER_WS_ID':'SCA_1_PLAYER_FBREF_ID'}, axis=1, inplace=True)

In [ ]:
df_shots.loc[df_shots['SCA_1_PLAYER_FBREF_NAME'] == 'Loum Ndiaye', 'SCA_1_PLAYER_FBREF_NAME'] = "Mamadou N'Diaye"

In [ ]:
df_shots.shape

In [ ]:
df_shots = df_shots.merge(df_player_match[['SCA_1_PLAYER_FBREF_ID','SCA_1_PLAYER_FBREF_NAME','MATCH_ID']], 
                          on=['SCA_1_PLAYER_FBREF_NAME','MATCH_ID'], how='left')

In [ ]:
relevant_inds = df_shots[['SCA_1_PLAYER_FBREF_NAME', 'MATCH_ID', 'ACTION_ID']].drop_duplicates().index

In [ ]:
df_shots = df_shots.loc[relevant_inds]

In [ ]:
# df_shots = df_shots.merge(df_players[['SCA_1_PLAYER_FBREF_ID','SCA_1_PLAYER_FBREF_NAME']], on='SCA_1_PLAYER_FBREF_NAME', how='left')

In [ ]:
# df_players.rename({'SCA_1_PLAYER_FBREF_NAME':'SCA_2_PLAYER_FBREF_NAME'}, axis=1, inplace=True)
# df_players.rename({'SCA_1_PLAYER_FBREF_ID':'SCA_2_PLAYER_FBREF_ID'}, axis=1, inplace=True)

df_player_match.rename({'SCA_1_PLAYER_FBREF_NAME':'SCA_2_PLAYER_FBREF_NAME'}, axis=1, inplace=True)
df_player_match.rename({'SCA_1_PLAYER_FBREF_ID':'SCA_2_PLAYER_FBREF_ID'}, axis=1, inplace=True)

In [ ]:
# df_shots = df_shots.merge(df_players[['SCA_2_PLAYER_FBREF_ID','SCA_2_PLAYER_FBREF_NAME']], on='SCA_2_PLAYER_FBREF_NAME', how='left')
df_shots = df_shots.merge(df_player_match[['SCA_2_PLAYER_FBREF_ID','SCA_2_PLAYER_FBREF_NAME','MATCH_ID']], 
                          on=['SCA_2_PLAYER_FBREF_NAME','MATCH_ID'], how='left')

In [ ]:
relevant_inds = df_shots[['SCA_2_PLAYER_FBREF_NAME', 'MATCH_ID', 'ACTION_ID']].drop_duplicates().index

In [ ]:
df_shots = df_shots.loc[relevant_inds]

In [ ]:
inds_to_drop = (df_shots[df_shots[['MATCH_ID', 'ACTION_ID']].duplicated()].index)

In [ ]:
inds_to_drop

In [ ]:
df_shots = df_shots.drop(inds_to_drop)

In [ ]:
df_shots = df_shots.drop(['SCA_1_PLAYER_FBREF_NAME', 'SCA_2_PLAYER_FBREF_NAME'], axis=1)

In [ ]:
df_shots.rename({'SCA_1_PLAYER_FBREF_ID':'SCA_1_PLAYER_WS_ID', 'SCA_2_PLAYER_FBREF_ID':'SCA_2_PLAYER_WS_ID'}, 
                axis=1, inplace=True)

In [ ]:
df_shots.replace({pd.NA: np.nan}, inplace=True)

In [ ]:
df_shots['XG'] = df_shots['XG'].astype(float)
df_shots['PSXG'] = df_shots['PSXG'].astype(float)
df_shots['DISTANCE'] = df_shots['DISTANCE'].astype(int)

In [ ]:
df_shots.shape

In [ ]:
upsert_to_snowflake("SHOT_EVENTS", df_shots, ['MATCH_ID', 'ACTION_ID'])

In [ ]:
## DF SHOTS NEEDS SOME CLEANING UP